In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/05 16:19:42 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.188.75 instead (on interface en0)
22/09/05 16:19:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/05 16:19:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/05 16:19:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
ori_transaction1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot').sort('order_datetime')
ori_transaction2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot').sort('order_datetime')
 #read the parquet ori_transaction
ori_transaction = ori_transaction1.union(ori_transaction2)
ori_transaction2


user_id,merchant_abn,dollar_value,order_id,order_datetime
14,89022158521,60.67672336066594,8053afb6-38b1-40c...,2021-08-28
8,49891706470,47.31755694261772,534645e3-bc80-41a...,2021-08-28
14946,49505931725,85.73423012207064,37b9fa86-8cf1-495...,2021-08-28
14938,53546629749,659.4359432661599,23519a0e-db53-4b8...,2021-08-28
14940,54705111450,100.87325296466584,a44bef39-26db-4da...,2021-08-28
14942,89536028092,45.310999359990895,0f24811f-a13b-4e2...,2021-08-28
14,43820812476,163.61700277818517,62175fef-d6f5-4bc...,2021-08-28
14936,24852446429,9.78847188379984,5a04b94c-c553-47c...,2021-08-28
4,86223953880,313.0216678687628,6161bda6-030c-42c...,2021-08-28
14938,98269572896,86.80997844773242,a79e1147-9922-4d9...,2021-08-28


In [23]:
from pyspark.sql import Window
import numpy as np
wind = Window.partitionBy('merchant_abn')
q1= F.expr('percentile_approx(dollar_value, 0.25)')
q3= F.expr('percentile_approx(dollar_value, 0.75)')
testq = ori_transaction2.withColumn('q1', q1.over(wind))
testq = testq.withColumn('q3', q3.over(wind))

In [24]:
testq

user_id,merchant_abn,dollar_value,order_id,order_datetime,q1,q3
10294,10023283211,212.58064262127309,2aba9ef2-6362-42e...,2021-08-28,115.49603272397582,299.2175437081959
2115,10023283211,67.1025437230857,5ff7c129-45dc-4af...,2021-08-29,115.49603272397582,299.2175437081959
1533,10023283211,287.40370605248404,f90c5485-29ac-4aa...,2021-08-30,115.49603272397582,299.2175437081959
8052,10023283211,491.34569919767677,7848fe2e-8293-4cb...,2021-08-30,115.49603272397582,299.2175437081959
9836,10023283211,20.422721659322292,64229c09-4d9d-4e5...,2021-08-30,115.49603272397582,299.2175437081959
17141,10023283211,111.7001685435339,938fb5e3-0374-440...,2021-08-31,115.49603272397582,299.2175437081959
16816,10023283211,135.21010094767303,ffbf7f69-5e1c-477...,2021-09-01,115.49603272397582,299.2175437081959
18283,10023283211,283.6950177030991,ebc6c6fb-67ef-484...,2021-09-01,115.49603272397582,299.2175437081959
19085,10023283211,200.19849282846627,0fb47b7e-1174-495...,2021-09-01,115.49603272397582,299.2175437081959
20024,10023283211,165.09769555368052,c0a132c4-acfb-40a...,2021-09-01,115.49603272397582,299.2175437081959


In [25]:
testq.tail(5)

[Row(user_id=23686, merchant_abn=99009287608, dollar_value=73.03735556043877, order_id='0bda9178-2377-4e78-ab80-3cde9892cb91', order_datetime=datetime.date(2022, 2, 17), q1=75.79010937115729, q3=251.69623955460847),
 Row(user_id=6097, merchant_abn=99009287608, dollar_value=45.978020704256686, order_id='6f93323d-e6a7-44d2-8283-b913bc641af1', order_datetime=datetime.date(2022, 2, 20), q1=75.79010937115729, q3=251.69623955460847),
 Row(user_id=19507, merchant_abn=99009287608, dollar_value=99.03551194564018, order_id='b3d4cd5f-f20c-43d8-9826-13a3a960039b', order_datetime=datetime.date(2022, 2, 22), q1=75.79010937115729, q3=251.69623955460847),
 Row(user_id=8197, merchant_abn=99009287608, dollar_value=48.66654072282729, order_id='046f358f-73a5-481b-a571-b7bbb60d9dbb', order_datetime=datetime.date(2022, 2, 23), q1=75.79010937115729, q3=251.69623955460847),
 Row(user_id=13904, merchant_abn=99009287608, dollar_value=96.09601595280076, order_id='584ddc64-05e4-455f-ae5a-e7a9d5295c06', order_date